# Implemtation of the graph

- Inicializar el grafo vacio NetworkX
- Crear un nodo. Para inicializar ese nodo buscamos en google (Primer resultado)
- N engines (computacion en paralelo) que busquen en los links
- Filtros URLs, Filtros Organizaciones (Revisar) - GPT2 (Clustering)
- Criterio de parada del agente.
- https://spacy.io/api/entityrecognizer


In [56]:
# Connection libraries.
import urllib.request, urllib.parse, urllib.error
import requests
from bs4 import BeautifulSoup
#from langdetect import detect
import re
import hashlib 
import time
import langdetect

import spacy
import es_core_news_sm
import en_core_web_sm
import networkx as nx

In [57]:
# Import class TheWitness (webscraper)
%run ./TheWitness.ipynb

# Import class Node 
%run ./Website.ipynb

#Import text clustering model
%run ./text_clustering_final.ipynb

In [58]:
def addNode(graph, result):
    website=Website(result[0], result[1],result[2],result[3],result[4], result[5],result[6], result[7],result[8],result[9],
                    result[10], result[11],result[12], result[13],result[14],result[15])
    graph.add_node(website)
    return len(list(graph.nodes))

In [59]:
def addEdge(graph, node1, node2):
    graph.add_edge(node1, node2)
    return(len(list(graph.nodes)))

In [60]:
def searchable_url(hostname):
    return "https://" + hostname

In [61]:
def get_text_(hostname):
    result = requests.get(searchable_url(hostname))
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    for script in soup(["script", "style"]):                   
            script.decompose()
    text = soup.get_text().replace("\n"," ")
    text = re.sub('\s+',' ',text)
    return text

In [62]:
def is_node_useful(possible_nodes, treshold):
    final_nodes = []
    for possible_node in possible_nodes:
        text = get_text_(searchable_url(possible_node))
        is_useful = cluster_text(text, treshold)
        if is_useful != -1:
            final_nodes.append(possible_node)
    return final_nodes

In [63]:
def index_of_node(graph, node):
    graph_list = list(graph)
    index = graph_list.index(node)
    return index

In [64]:
# def explore(graph, predecesor_node_url, node_url, prev_witness)
#     if prev_witness.getDistance() > 3:
#         pass
#     else:
#         for node in toVisit_nodes:
#             witness = TheWitness(predecesor_node_url, node_url, index_of_node(graph, node_url), witness.getDistance()+1)
#             result = witness.scrap()
#             addNode(graph, result)
#             addEdge(graph, predecesor_node, node)

#             for granchild_node in result[4]:
#                 explore(graph, node, distance_from_root + 1, )

In [65]:
def explore(node, witness):
    if witness.getDistance()<3:
        
        useful_nodes = is_node_useful(node.getUrls(), global_treshold)
        for n in useful_nodes:
            
            w=TheWitness(node.getUrl(), n, 1, witness.getDistance()+1)
            try:
                result = w.scrap()
                node_pos=addNode(result)-1
                node1=list(G.nodes)[node_pos]
                addEdge(graph, node, node1)
                del witness
                explore(node1, w)
            except:
                del witness
                pass

In [66]:
G = nx.Graph()

In [67]:
def explore_root(root_url):
    witness_root = TheWitness(root_url, root_url, 0, 0)
    result_root = witness_root.scrap()
    node_root_pos = addNode(G,result_root)-1
    node_root = list(G.nodes)[node_root_pos]
    
    explore(node_root, witness_root)

In [70]:
root_url = "https://co.mileroticos.com/"
explore_root(root_url)

ConnectionError: HTTPSConnectionPool(host='https', port=443): Max retries exceeded with url: //mx.mileroticos.com (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F1EC2F0808>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

# Final

In [ ]:
def final_algorithm():
    G = nx.Graph()

In [6]:
# For link in links.
def explore_network(result):
    
    count = 1
    for link in result[2]: 

        
        # We already explored this website.
        try: 
            if hashes_map[link]: 
                pass

        # New website!
        except:
            
            navegable = True
            for website in omit_pages: 
                if link.find(website) != -1: 
                    navegable = False
                    

            if navegable: 
                root = "https://modelosalacarta.com/"
                destination = link
                hash_destination = nodeID(destination)
                distance_from_root = 1
                #hashes_map[destination] = True

                witness = TheWitness(
                    root, 
                    destination, 
                    hash_destination, 
                    distance_from_root
                )
                result = witness.scrap()
                
                print("---------------")
                print("Agent No.", count)
                print(result)
                print("---------------")
                
                del witness
                
        count += 1